In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [ ]:
gamma = 0.999
alpha = 1
door = -4
wall = -200
home = 1000

In [ ]:
rewardsTable = np.array([
    [None,-1,None,None,wall,None,None,None,None,None,None,None,None,None,None,None],
    [-1,None,-1,None,None,door,None,None,None,None,None,None,None,None,None,None],
    [None,-1,None,-1,None,None,wall,None,None,None,None,None,None,None,None,None],
    [None,None,-1,None,None,None,None,-1,None,None,None,None,None,None,None,None],
    [-1,None,None,None,None,door,None,None,-1,None,None,None,None,None,None,None],
    [None,-1,None,None,wall,None,wall,None,None,-1,None,None,None,None,None,None],
    [None,None,-1,None,None,door,None,-1,None,None,-1,None,None,None,None,None,None],
    [None,None,None,-1,None,None,wall,None,None,None,None,-1,None,None,None,None],
    [None,None,None,None,wall,None,None,None,None,-1,None,None,-1,None,None,None],
    [None,None,None,None,None,door,None,None,-1,None,-1,None,None,home,None,None],
    [None,None,None,None,None,None,wall,None,None,-1,None,-1,None,None,-1,None],
    [None,None,None,None,None,None,None,-1,None,None,-1,None,None,None,None,-1],
    [None,None,None,None,None,None,None,None,-1,None,None,None,None,home,None,None],
    [None,None,None,None,None,None,None,None,None,-1,None,None,-1,None,-1,None],
    [None,None,None,None,None,None,None,None,None,None,-1,None,None,home,None,-1],
    [None,None,None,None,None,None,None,None,None,None,None,-1,None,None,-1,None]
    ], dtype='object')

In [ ]:
coordinatesToStates = {
    'A1' : 0,
    'B1' : 1,
    'C1' : 2,
    'D1' : 3,
    'A2' : 4,
    'B2' : 5,
    'C2' : 6,
    'D2' : 7,
    'A3' : 8,
    'B3' : 9,
    'C3' : 10,
    'D3' : 11,
    'A4' : 12,
    'B4' : 13,
    'C4' : 14,
    'D4' : 15,
}


In [ ]:
possibleActions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]


In [ ]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

def policy(stateT, legalActions, QValues, method, epsilon):
    if method == 'QLearning':
        return np.random.choice(legalActions)
    if method == 'SARSA':
        if np.random.random() < epsilon:
            return np.random.choice(legalActions)
        else:
            bestAction = legalActions[0]
            for index in legalActions:
                if QValues[stateT][index] >= QValues[stateT][bestAction]:
                    bestAction = index
            return bestAction

def legalActionChecker(index, rewards, state):
    legalActions = []
    i = 0
    while i <= index:
        if rewards[state][i] != None:
            legalActions.append(i)
        i+=1
    return legalActions

def argmaxAction(legalAction, QTable, stateT):
    maxQAction = legalAction[0]
    for index in legalAction:
        if QTable[stateT][index] >= QTable[stateT][maxQAction]:
            maxQAction = index
    return maxQAction
    
def path(QValues, startCoord = 'A1', endCoord = 'B4'):
    path = [startCoord]
    reward = 0
    nextCoord = startCoord
    while (nextCoord != endCoord):
        startState = coordinatesToStates[startCoord]
        legalActions = legalActionChecker(15, rewardsTable, startState)
        nextState = argmaxAction(legalActions, QValues, startState)
        nextCoord = stateToCoordinates[nextState]
        reward += rewardsTable[startState][nextState]
        path.append(nextCoord)
        startCoord = nextCoord
    return {'Reward': reward, 'Path':path}

def qLearning(episodeCount, method, epsilon):
    currentEpisode = 0
    QValues = np.zeros((16, 16))

    while currentEpisode < episodeCount:
        stateT = 0
        stateNext = int
        legalActions = []
        reward = 0
        rewardArr = []
        i = 0
        #While haven't reached the terminal state
        while stateNext != 13:
            legalActions = legalActionChecker(15,rewardsTable,stateT)
            #print('Step',i, 'is ', stateT)
            
            stateNext= policy(stateT, legalActions,QValues,method,epsilon)
            reward += rewardsTable[stateT][stateNext]
            bestAction = argmaxAction(legalActionChecker(15,rewardsTable,stateNext), QValues, stateNext)
            TD = rewardsTable[stateT][stateNext] + gamma * QValues[stateNext][bestAction] - QValues[stateT][stateNext]
            QValues[stateT][stateNext] += alpha*TD 
            stateT=stateNext
            legalActions = []
            i = 0
        rewardArr.append(path(QValues)['Reward'])
        reward = 0
        print(rewardArr)
        currentEpisode += 1
    return {'QValues': QValues, 'rewardArr': rewardArr}


In [ ]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

In [ ]:
#pd.DataFrame(qLearning(1,'QLearning', 0.5).astype(int), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

qlearn = qLearning(100,'QLearning', 0.5)
#print(qLearning(15,'SARSA', 0.5).astype(int))
print(path(qlearn['QValues']))
plt.plot(qlearn['rewardArr'])


In [ ]:
#pd.DataFrame(qLearning(1,'SARSA', 0.4), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
